In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd 
from urllib.parse import quote

In [2]:
import unicodedata
import re
from tqdm.notebook import tqdm

In [3]:
insta_url = 'https://www.instagram.com'
driver = webdriver.Chrome('chromedriver')
driver.get(insta_url)
time.sleep(1)

In [4]:
with open('password.txt') as f:
    password = f.read()

In [5]:
email = 'minu7972'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [6]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [7]:
# 로그인 정보 저장 - 나중에 하기
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [8]:
# 알림 설정 - 나중에 하기
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

In [9]:
keyword = '제주맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%A7%9B%EC%A7%91'

In [10]:
driver.get(url)
time.sleep(5)

# 첫번째 게시글

In [12]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)


In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content


In [ ]:
tags = re.findall(r'#[^\s#,\\]+', content)
tags

In [ ]:
# 작성일자 정보 가져오기
date = soup.select_one('time._1o9PC.Nzb55')['datetime'][:10]
date

# 5) 위치정보 가져오기

In [ ]:
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

In [ ]:
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

In [ ]:
row = [content, date, like, place, tags]
row


In [ ]:
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
time.sleep(2)

In [ ]:
# 두번째 게시글부터 다음글로 이동
driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
time.sleep(2)

In [ ]:
# 5를 하기 위해 popup 화면을 없앰(x버튼 누르기)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()

# 5반복문으로 가져오기

In [13]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, date, like, place, tags]


In [ ]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [14]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div[2]/button').click()
        time.sleep(5)
    except:
        time.sleep(5)
        driver.find_element_by_xpath('/html/body/div[6]/div[1]/div/div/div/button').click()
        time.sleep(5)

  0%|          | 0/500 [00:00<?, ?it/s]

In [23]:
df = pd.DataFrame(results,columns=['내용','날짜','조회수','장소','테그'])
df.head(20)

,내용,날짜,조회수,장소,테그
0,"제주 갈치요리하면 요기 !!!비주얼 최고맛집이라불리는 갈치요리맛집""성산을본갈치""성산...",2021-12-07,0,,[#성산을본갈치주소]
1,광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 7...,2021-12-06,0,,"[#애월맛집, #애월카페, #애월읍맛집, #제주애월맛집, #제주도맛집, #제주맛집,..."
2,#리그램 - @gunwoo_o942021. 11. 30 제주 1일❤️..또 가고싶다...,2021-12-07,328,월정리해변,"[#리그램, #안녕하샌]"
3,"#제주맛집산도롱맨도롱----------🍜갈비국수 ""찐"" 맛집✔💕 큼직한 우대갈비가 ...",2021-12-08,0,산도롱맨도롱,"[#제주맛집산도롱맨도롱----------🍜갈비국수, #제주도맛집.대기시간이, #제주..."
4,.💖드르쿰다in성산💖아주 크게 펼쳐진 초대형 스튜디오에서이쁜 사진을 찍고 낮에는 커...,2021-12-07,"1,030",,"[#성산카페, #성산맛집, #제주카페, #제주도카페, #제주여행, #제주핫플, #제..."
5,영업시간 네이버 참고 #제주맛집 #제주도맛집#에코랜드 #사려니숲길 #코코몽...,2021-12-07,140,,"[#제주맛집, #제주도맛집, #에코랜드, #사려니숲길, #코코몽에코파크제주, #스누..."
6,"🤍⁣⁣🥩 애월 바다뷰 맛집 ""애월리식당"" 🥩⁣⁣별점 - 🌟🌟🌟🌟🌟 x10000000...",2021-12-07,0,,"[#제주맛집, #애월흑돼지, #애월흑돼지맛집, #애월흑돼지맛집추천, #애월맛집인정,..."
7,"🤍⁣⁣❣ 제주 서귀포 회맛집, ""김선장회센타"" ❣⁣⁣김선장SET으로 즐기는 다양하고...",2021-12-07,0,,"[#제주맛집, #제주회맛집, #제주회포장, #서귀포핫플, #서귀포횟집, #제주도횟집..."
8,#제주#제주도#제주여행#신창해안도로#싱계물공원#판포포구#제주맛집#제주도맛집#한경면맛...,2021-12-08,270,,"[#제주, #제주도, #제주여행, #신창해안도로, #싱계물공원, #판포포구, #제주..."
9,"#오만과자점 제주 동쪽 오만과자점✨12월 첫째주, 둘째주는답례품, 대량 주문 마감입...",2021-12-08,52,하도해변,"[#오만과자점, #제주쿠키, #제주디저트카페, #제주동쪽, #구좌읍카페, #구좌읍맛..."


In [20]:
df.drop_duplicates(subset=df.내용,inplace=True)

KeyError: Index(['"두부" 첫 오픈일!!!❤️.********************************"두부" 제주 한림읍 협재 10길 14********************************@dubu_food.drink.stay .#두부#두부없는두부집#제주맛집두부#제주커피맛집 #제주와인맛집#와인맛집#협재맛집 #제주맛집#협재숙소#감성숙소#제주숙소#커플민박#커플펜션#제주#협재해수욕장 #비양도#jeju#jejuisland#coffee#wine#pasta#브런치#제주브런치#제주브런치카페 #제주와플맛집',
       '"두부" 첫 오픈했습니다!놀러오세요!❤️.********************************"두부" 제주 한림읍 협재 10길 14********************************@dubu_food.drink.stay .#두부#두부없는두부집#제주맛집두부#제주커피맛집 #제주와인맛집#와인맛집#협재맛집 #제주맛집#협재숙소#감성숙소#제주숙소#커플민박#커플펜션#제주#협재해수욕장 #비양도#jeju#jejuisland#coffee#wine#pasta#브런치#제주브런치#제주브런치카페 #제주와플맛집',
       '#3일차호텔빠이#펜션하잉#제주#힐링#힐링은휴식#버버리제주#부부스타그램❤️#럽스타그램💕#목말라#눈뜨자마자맥주#또땡긴다#한라산#정복.......#제주도에서살기#제주아일랜드#제주맛집#제주탕진#탕진잼#셋째날도탕진각#고고#레고레고#눈뜨자마자👀#술스타그램#물1도안마시고술만마시는중#부어라마셔라취하자#흥하자!#다들감사합니다',
       '#감따남.안녕하세요 감따남을 운영하는 대표🍊 김준석입니다^^.많은 분들의 성원과 관심으로 지속적으로성장할 수 있었습니다~ 정말 감사드립니다 :D.할아버지때부터 3대째 이어져오고 있는감따남 착한농장(감귤,황금향,레드향,한라봉)체험, 전국유통)제주핫플카페 감따남, 제주인기스냅 감따남스냅을운영하면서 쌓인 노하우들을 홍보가 필요하시거나컨설팅이 필요하신 업체 대표님들에게 컨설팅하고홍보하여 드리고자 '감따남

In [19]:
df.내용

0      제주 갈치요리하면 요기 !!!비주얼 최고맛집이라불리는 갈치요리맛집"성산을본갈치"성산...
1      광고제주도에서 즐기는 랍스터만찬 랍스터빈 !!🔸주소 : 제주시 애월읍 애월해안로 7...
2      #리그램 - @gunwoo_o942021. 11. 30 제주 1일❤️..또 가고싶다...
3      #제주맛집산도롱맨도롱----------🍜갈비국수 "찐" 맛집✔💕 큼직한 우대갈비가 ...
4      .💖드르쿰다in성산💖아주 크게 펼쳐진 초대형 스튜디오에서이쁜 사진을 찍고 낮에는 커...
                             ...                        
446    ☕️ 카페 쪼끌락 ☕️달달한 돌하르방 라떼가 유명한 김녕 카페🤍+ 뷰까지 완벽한 곳...
447    ☕️ 카페 쪼끌락 ☕️달달한 돌하르방 라떼가 유명한 김녕 카페🤍+ 뷰까지 완벽한 곳...
448    ☕️ 카페 쪼끌락 ☕️달달한 돌하르방 라떼가 유명한 김녕 카페🤍+ 뷰까지 완벽한 곳...
449    ☕️ 카페 쪼끌락 ☕️달달한 돌하르방 라떼가 유명한 김녕 카페🤍+ 뷰까지 완벽한 곳...
450    ☕️ 카페 쪼끌락 ☕️달달한 돌하르방 라떼가 유명한 김녕 카페🤍+ 뷰까지 완벽한 곳...
Name: 내용, Length: 451, dtype: object

In [24]:
driver.close()